In [1]:
include("inference/mcmc/sepsis_types.jl")
using .SepsisTypes
include("inference/mcmc/sepsis.jl")
using .Sepsis
include("inference/mcmc/inference.jl")
using .Inference
using Revise
using PyCall;
using Gen;
using CairoMakie
sepsis_gym = pyimport("custom_sepsis");

┌ Warning: attempting to remove probably stale pidfile
│   path = /Users/luisastue/.julia/compiled/v1.10/CairoMakie/9mSey_ZUpiO.ji.pidfile
└ @ FileWatching.Pidfile /Users/luisastue/.julia/juliaup/julia-1.10.6+0.aarch64.apple.darwin14/share/julia/stdlib/v1.10/FileWatching/src/pidfile.jl:244


In [2]:
policy = sepsis_gym.random_policy()
episode = sepsis_gym.run_episode(policy)

PyObject <custom_sepsis.sepsis_env.sepsis_types.Episode object at 0x12f023c40>

In [3]:
to_state(episode.visited[1])


State(NORMAL, LOW, NORMAL, SUPER_HIGH, true, false, false, false)

In [ ]:
beliefs = get_beliefs()
init_state = to_state(sepsis_gym.random_initial_state())
states, rewards = simulate_episode(beliefs, to_policy(policy), init_state)
rewards

get_reward(states[end])

In [ ]:
to_state(episode.visited[1])

State(NORMAL, LOW, LOW, SUPER_HIGH, false, false, false, false)

In [ ]:
println("Checking if the actions in the episode are consistent with the state transitions")
for (i, state) in enumerate(episode.visited[1:end-1])
    action = to_action(policy[state])
    next_state = to_state(episode.visited[i+1])
    println(action.abx == next_state.abx && action.vaso == next_state.vaso && action.vent == next_state.vent)
end

In [ ]:
env_params = get_beliefs()
param_vector = get_env_params(env_params)


In [ ]:
policy = sepsis_gym.random_policy()
sepsis_gym.run_episode(policy).rewards

env = sepsis_gym.SepsisEnv(param_vector)

training = sepsis_gym.train_dqn(true_env, 10000, "test_call_from_julia")
training.evaluate_policy(10000)

UndefVarError: UndefVarError: `param_vector` not defined

In [ ]:
random_policies = [sepsis_gym.random_policy() for i in 1:100];

In [ ]:
sepsis_gym.run_episode(random_policies[1]).visited


2-element Vector{Tuple{Int64, Int64, Int64, Int64, Vararg{Bool, 4}}}:
 (-1, -1, 0, 1, 0, 0, 0, 0)
 (-1, -1, 0, 1, 0, 0, 0, 0)

In [ ]:
# param_vector = get_env_params(traces[end][:beliefs])

# env = sepsis_gym.SepsisEnv(param_vector)

# training = sepsis_gym.train_dqn(env, 100000, "thompson_sampling_try1")


In [ ]:
# training.evaluate_policy(10000)

In [4]:

state = State(NORMAL, LOW, NORMAL, NORMAL, false, false, false, false)
deterministic_params = EnvParameters([1 for _ in 1:length(fieldnames(EnvParameters))]...)
policy = to_policy(sepsis_gym.random_policy())
next_state = get_next_state(deterministic_params, state, policy[state])
trace, sc = generate(get_next_state, (deterministic_params, state, Action(0, 0, 0)))

states, rewards = simulate_episode(deterministic_params, policy, state)
states

UndefVarError: UndefVarError: `EnvParameters` not defined